In [1]:
import re
import numpy as np
import pandas as pd

In [2]:
data=pd.read_csv(r'/Users/liqinglin/Desktop/python/document/data.csv')

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [12]:
data.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [14]:
data.shape[0]

406829

In [11]:
data.dropna(subset=['CustomerID'],axis='index',inplace=True)

In [15]:
data.drop_duplicates(keep='first',inplace=True)

In [16]:
data.shape[0]

401604

In [17]:
COLUMN_NAME_MAP = {"InvoiceNo": "invoice_no",
                   "StockCode": "stock_code",
                   "Description": "description",
                   "Quantity": "quantity",
                   "InvoiceDate": "invoice_date",
                   "UnitPrice": "unit_price",
                   "CustomerID": "customer_id",
                   "Country": "country"}

In [19]:
df=data.rename(columns=COLUMN_NAME_MAP)

In [20]:
df.head(2)

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,customer_id,country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [25]:
product1=df.groupby(["stock_code","description"]).agg(
    min_price=('unit_price','min'),
    max_price=('unit_price','max'),
    median_price=('unit_price','median'),
    std_price=('unit_price','std'),
    mean_price=('unit_price','mean'),
    count=('unit_price','count')
    )

In [28]:
product1.size

23496

In [29]:
product1.query(f'max_price>{np.percentile(product1.max_price,99)}').sort_values('max_price',ascending=False)

,,min_price,max_price,median_price,std_price,mean_price,count
stock_code,description,,,,,,
M,Manual,0.00,38970.00,3.250,1945.267761,338.044391,460
POST,POSTAGE,1.00,8142.75,18.000,334.121790,37.888528,1196
D,Discount,0.01,1867.86,22.970,219.271071,72.484545,77
DOT,DOTCOM POSTAGE,11.17,1599.26,715.850,540.829708,744.147500,16
CRUK,CRUK Commission,1.60,1100.44,471.770,364.164786,495.839375,16
22502,PICNIC BASKET WICKER 60 PIECES,649.50,649.50,649.500,0.000000,649.500000,2
22655,VINTAGE RED KITCHEN CABINET,50.00,295.00,125.000,67.163983,150.663043,46
22656,VINTAGE BLUE KITCHEN CABINET,50.00,295.00,125.000,58.575724,143.653846,26
22826,LOVE SEAT ANTIQUE WHITE METAL,42.50,195.00,85.000,50.800287,115.388889,45


In [30]:
def get_stock_type(x):
    if re.match("\d{5}[A-Z]?",str(x)):#0-5个数字 A-Z 字母0个或者1个
        return "normal"
    elif re.match("DCGS.+?",str(x)):
        return "DCGS"
    elif re.match("gift.+?",str(x)):
        return "gift"
    else:
        return "others"
df=df.assign(stock_type=df['stock_code'].map(get_stock_type),is_zero_price=(df.unit_price==0)*1)

In [31]:
df.tail()

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,customer_id,country,stock_type,is_zero_price
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France,normal,0
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France,normal,0
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France,normal,0
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France,normal,0
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,12/9/2011 12:50,4.95,12680.0,France,normal,0


re.sub()：匹配替换为选择的文本。

In [60]:
df.customer_id=df.customer_id.astype(str).str.rstrip('0').str.rstrip('.')

In [62]:
df.customer_id.value_counts()

17841    7812
14911    5898
14096    5128
12748    4459
14606    2759
         ... 
17102       1
18113       1
16738       1
15590       1
17956       1
Name: customer_id, Length: 4372, dtype: int64

0         1785
1         1785
2         1785
3         1785
4         1785
          ... 
541904    1268
541905    1268
541906    1268
541907    1268
541908    1268
Name: customer_id, Length: 401604, dtype: object

In [63]:
df.invoice_no.value_counts()

576339     542
579196     533
580727     529
578270     442
573576     435
          ... 
C552995      1
C578003      1
553099       1
C564204      1
C547988      1
Name: invoice_no, Length: 22190, dtype: int64

In [64]:
mdf=df.assign(
    is_cancel=(df.invoice_no.str.contains('C'))*1,
    invoice_date=pd.to_datetime(df.invoice_date))

In [66]:
mdf.is_cancel.value_counts()

0    392732
1      8872
Name: is_cancel, dtype: int64

In [68]:
mdf=mdf.assign(
    invoice_day=mdf.invoice_date.dt.date,
    invoice_hour=mdf.invoice_date.dt.hour)

In [119]:
mdf = mdf.assign(
    amount=mdf["quantity"] * mdf["unit_price"],
    weekday = mdf["invoice_day"].map(lambda x:str(x.isoweekday())),
    invoice_year_month = mdf["invoice_day"].map(lambda x :f"{x.year}-{x.month:02}"),
    invoice_year_week = mdf["invoice_day"].map(lambda x :f"{x.isocalendar()[0]}-{x.isocalendar()[1]:02}")
)

In [116]:
mdf.head()

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,customer_id,country,stock_type,is_zero_price,is_cancel,invoice_day,invoice_hour,amount,weekday,invoice_year_month,invoice_year_week
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,normal,0,0,2010-12-01,8,15.30,3,2010-12,2010-48
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,normal,0,0,2010-12-01,8,20.34,3,2010-12,2010-48
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,normal,0,0,2010-12-01,8,22.00,3,2010-12,2010-48
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,normal,0,0,2010-12-01,8,20.34,3,2010-12,2010-48
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,normal,0,0,2010-12-01,8,20.34,3,2010-12,2010-48


In [114]:
mdf['invoice_day'].map(lambda x: x.isoweekday())

0         3
1         3
2         3
3         3
4         3
         ..
541904    5
541905    5
541906    5
541907    5
541908    5
Name: invoice_day, Length: 401604, dtype: int64

In [118]:
mdf.dtypes

invoice_no                    object
stock_code                    object
description                   object
quantity                       int64
invoice_date          datetime64[ns]
unit_price                   float64
customer_id                   object
country                       object
stock_type                    object
is_zero_price                  int64
is_cancel                      int64
invoice_day                   object
invoice_hour                   int64
amount                       float64
weekday                       object
invoice_year_month            object
invoice_year_week             object
dtype: object

In [120]:
mdf.to_csv(r'/Users/liqinglin/Desktop/python/document/gift_ecom.csv',index=False,encoding='UTF-8')

In [123]:
mdf.stock_type.value_counts()

normal    399689
others      1915
Name: stock_type, dtype: int64

In [1]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt 

In [3]:
import pandas as pd

In [5]:
mdf=pd.read_csv(r'/Users/liqinglin/Desktop/python/document/gift_ecom.csv')

In [6]:
word_cloud = WordCloud(
                       background_color="white", # 词云图的背景颜色
                       ) # 去掉的停词
word_cloud.generate(mdf.description) 

TypeError: expected string or bytes-like object

In [47]:
mds=mdf.groupby('stock_code')['amount'].sum()
mds=mds.reset_index().rename(columns={'amount':'amo'})

In [53]:
mdn=pd.merge(mdf,mds,how='left',left_on='stock_code',right_on='stock_code')

In [49]:
mds.head()

,stock_code,amo,arank
0,10002,699.55,1965.0
1,10080,114.41,898.0
2,10120,40.32,543.0
3,10123C,3.25,90.0
4,10124A,6.72,160.5


In [44]:
mdn['amo_rank']=mdn['amo'].rank(ascending=False)

In [39]:
mdn.arank.sort_values(ascending=True).head()

98253     1.0
98258     1.0
98259     1.0
98260     1.0
204528    1.0
Name: arank, dtype: float64

In [52]:
mds['arank']=mds.amo.rank(ascending=False)

In [55]:
mdn.to_csv(r'/Users/liqinglin/Desktop/python/document/gift_ecom.csv',index=False,encoding='UTF-8')

In [54]:
mdn.head()

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,customer_id,country,stock_type,is_zero_price,is_cancel,invoice_day,invoice_hour,amount,weekday,invoice_year_month,invoice_year_week,amo,arank
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,normal,0,0,2010-12-01,8,15.30,3,2010-12,2010-48,93923.15,2.0
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,normal,0,0,2010-12-01,8,20.34,3,2010-12,2010-48,5753.68,374.0
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,normal,0,0,2010-12-01,8,22.00,3,2010-12,2010-48,6958.73,286.0
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,normal,0,0,2010-12-01,8,20.34,3,2010-12,2010-48,9255.17,201.0
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,normal,0,0,2010-12-01,8,20.34,3,2010-12,2010-48,13234.81,118.0


In [ ]:
mdn